


.




## BlazingContext
[Docs](https://docs.blazingdb.com/docs/blazingcontext) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#BlazingContext)

In [ ]:
from blazingsql import BlazingContext

Create basic BlazingContext instance.

In [ ]:
bc = BlazingContext()

Configure some options when creating a BlazingContext instance.

In [ ]:
bc = BlazingContext(dask_client=None,
                    network_interface=None,
                    allocator='managed',
                    pool=False,
                    initial_pool_size=None,
                    enable_logging=False,
                    config_options={})

.


.


.

### Distributed SQL
[Docs](https://docs.blazingdb.com/docs/distributed) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#Distributed-SQL)

Distribute BlazingSQL query execution across multiple GPUs with Dask. 

#### Single Node Multi-GPU
Distribute BlazingSQL query execution across all GPUs within the same system. 

Read more with our blog post "[Distributed SQL with Dask](https://blog.blazingdb.com/distributed-sql-with-dask-2979262acc8a?source=friends_link&sk=077319064cd7d9e18df8c0292eb5d33d)".

In [ ]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

cluster = LocalCUDACluster()
client = Client(cluster)

In [ ]:
from blazingsql import BlazingContext

bc = BlazingContext(dask_client=client, network_interface='lo')

.


.


.

## Storage Plugins

[Docs](https://docs.blazingdb.com/docs/connecting-data-sources) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#Storage-Plugins)


In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()

Register public AWS S3 bucket.

In [ ]:
bc.s3('bsql', bucket_name='blazingsql-colab')

.


.


.

## Create Tables
[Docs](https://docs.blazingdb.com/docs/creating-tables) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#Create-Tables)


In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()

Create table from local CSV file.

In [ ]:
bc.create_table('taxi', '../data/sample_taxi.csv', header=0)

Create table from local Parquet file.

In [ ]:
bc.create_table('taxi', '../data/sample_taxi.parquet')

Create table from Parquet file stored in public AWS S3 bucket.

In [ ]:
bc.s3('bsql', bucket_name='blazingsql-colab')

bc.create_table('taxi', 's3://bsql/yellow_taxi/taxi_data.parquet')

.


.


.

## Query Tables
[Docs](https://docs.blazingdb.com/docs/single-gpu) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#Create-Tables)

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()

In [ ]:
bc.create_table('taxi', '../data/sample_taxi.parquet')

Pull all rows and all columns.

In [ ]:
bc.sql('SELECT * FROM taxi')

Determine the average number riders per trip by hour of the day.

In [ ]:
avg_riders_by_hour = '''
                     select
                         avg(cast(passenger_count as float)) as avg_passenger_count,
                         hour(dropoff_ts) as hour_of_the_day
                     from (
                         select
                             passenger_count, 
                             cast(tpep_dropoff_datetime || '.0' as timestamp) dropoff_ts
                         from
                             taxi
                             )
                     group by
                         hour(dropoff_ts)
                     order by
                         hour(dropoff_ts)
                         '''
bc.sql(avg_riders_by_hour)

.


.


.

# SQL Reference




.




## Data Types
[Docs](https://docs.blazingdb.com/docs/database-administration) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#SQL-Reference)

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()

In [ ]:
import cudf
df = cudf.read_csv('../data/sample_taxi.csv')

These are the currently supported data types for BlazingSQL:

    Integer
        INT8
        INT16
        INT32
        INT64

In [ ]:
df['VendorID'] = df['VendorID'].astype('int8')

df['passenger_count'] = df['passenger_count'].astype('int16')

df['trip_distance'] = df['trip_distance'].astype('int32')

df['RatecodeID'] = df['RatecodeID'].astype('int64')

bc.create_table('taxi', df)

In [ ]:
bc.sql('SELECT VendorID, passenger_count, trip_distance, RatecodeID FROM taxi LIMIT 5')


#### Floating Point
        FLOAT8
        FLOAT16
        FLOAT32
        FLOAT64

In [ ]:
df['RatecodeID'] = df['RatecodeID'].astype('float32')

df['payment_type'] = df['payment_type'].astype('float64')

bc.create_table('taxi', df)

In [ ]:
bc.sql('SELECT RatecodeID, payment_type FROM taxi LIMIT 5')

#### Strings
        STR
#### Date Time
        DATE32 (INT32_T DAYS since the UNIX Epoch)
        DATE64 (INT64_T MILLISECONDS since the UNIX Epoch)
        TIMESTAMP (INT64 encoded as exact TIMESTAMP since the UNIX Epoch)

In [ ]:
df['tpep_pickup_datetime'] = df['tpep_pickup_datetime'].astype('str')

df['dropoff_datetime64'] = df['tpep_dropoff_datetime'].astype('datetime64[ns]')

bc.create_table('taxi', df)

In [ ]:
query = '''
        select
            tpep_pickup_datetime,
            dropoff_datetime64,
            cast(tpep_dropoff_datetime as TIMESTAMP)
        from
            taxi
            '''
bc.sql(query)

.


.


.

## General SQL
[Docs](https://docs.blazingdb.com/docs/queries) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#General-SQL)

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()

In [ ]:
bc.create_table('taxi', '../data/sample_taxi.parquet')

### SELECT
[Docs](https://docs.blazingdb.com/docs/queries#select) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#SELECT)

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

In [ ]:
bc.sql('SELECT * from taxi')

In [ ]:
bc.sql('SELECT passenger_count, trip_distance, fare_amount from taxi')

### WHERE
[Docs](https://docs.blazingdb.com/docs/queries#where) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#WHERE)

Query data and filter rows with a condition(s).

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

In [ ]:
bc.sql('select * from taxi WHERE passenger_count = 2')

#### WHERE -- Multiple Filters (AND)

In [ ]:
bc.sql('select * from taxi WHERE passenger_count > 2 AND trip_distance < 10')

#### WHERE -- Multiple Filters (OR)

In [ ]:
bc.sql('select * from taxi WHERE passenger_count >= 2 OR trip_distance <= 10')

#### WHERE -- Multiple Filters (AND/OR)

In [ ]:
query = ''' 
        select 
            * 
        from 
            taxi 
                WHERE 
                    (passenger_count <> 2 OR fare_amount > 100) 
                    AND trip_distance >= 10
                    '''
bc.sql(query)

### GROUP BY
[Docs](https://docs.blazingdb.com/docs/queries#group-by) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#GROUP-BY)

Groups result set of a query by one or more columns.

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

In [ ]:
bc.sql('select passenger_count from taxi GROUP BY passenger_count')

In [ ]:
query = '''
        select
            passenger_count,
            VendorID
        from
            taxi
        GROUP BY
            passenger_count, 
            VendorID
            '''
bc.sql(query)

#### SUM

In [ ]:
# --SUM without GROUP BY
bc.sql('select SUM(fare_amount) from taxi')

In [ ]:
# --SUM with GROUP BY
bc.sql('select SUM(fare_amount) from taxi GROUP BY passenger_count')

#### AVG

In [ ]:
# --AVG without GROUP BY
bc.sql('select AVG(fare_amount) from taxi')

In [ ]:
# --AVG with GROUP BY
bc.sql('select AVG(fare_amount) from taxi GROUP BY passenger_count')

#### MIN

In [ ]:
# --MIN without GROUP BY
bc.sql('select MIN(fare_amount) FROM taxi')

In [ ]:
# --MIN with GROUP BY
bc.sql('select MIN(fare_amount) from taxi GROUP BY passenger_count')

#### MAX

In [ ]:
# --MAX without GROUP BY
bc.sql('select MAX(fare_amount) FROM taxi')

In [ ]:
# --MAX with GROUP BY
bc.sql('select MAX(fare_amount) from taxi GROUP BY passenger_count')

#### COUNT

In [ ]:
# --COUNT without GROUP BY
bc.sql('select COUNT(fare_amount) from taxi')

In [ ]:
# --COUNT with GROUP BY
bc.sql('select COUNT(fare_amount) FROM taxi GROUP BY passenger_count')

#### DISTINCT

In [ ]:
# --COUNT without GROUP BY
bc.sql('select DISTINCT(passenger_count) from taxi')

In [ ]:
# --COUNT with GROUP BY
bc.sql('select DISTINCT(passenger_count) from taxi GROUP BY passenger_count')

### JOIN
[Docs](https://docs.blazingdb.com/docs/queries#join) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#JOIN)

Combine rows from two or more tables, based on a related column or columns between them.

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi_a', '../data/sample_taxi.parquet')
bc.create_table('taxi_b', '../data/sample_taxi.csv', header=0)

#### INNER JOIN
[Docs](https://docs.blazingdb.com/docs/queries#inner) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#INNER-JOIN)

#### INNER JOIN SINGLE COLUMN

In [ ]:
query = '''
        select 
            A.tpep_pickup_datetime, A.VendorID,
            B.passenger_count, B.trip_distance, B.fare_amount
        from 
            taxi_a as A
        INNER JOIN 
            taxi_b as B
            ON A.tpep_pickup_datetime = B.tpep_pickup_datetime
            '''
bc.sql(query)

#### INNER JOIN MULTI-COLUMN

In [ ]:
query = '''
        select 
            A.tpep_pickup_datetime, A.VendorID,
            B.passenger_count, B.trip_distance, B.fare_amount
        from 
            taxi_a as A
        INNER JOIN 
            taxi_b as B
            ON A.tpep_pickup_datetime = B.tpep_pickup_datetime
            AND A.VendorID = B.VendorID
            '''
bc.sql(query)

#### FULL OUTER JOIN
[Docs](https://docs.blazingdb.com/docs/queries#full-outer) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#FULL-OUTER-JOIN)

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi_a', '../data/sample_taxi.parquet')
bc.create_table('taxi_b', '../data/sample_taxi.csv', header=0)

#### FULL OUTER JOIN SINGLE COLUMN

In [ ]:
query = '''
        select 
            taxi_a.tpep_pickup_datetime, 
            taxi_b.passenger_count, 
            taxi_b.fare_amount 
        from 
            taxi_a 
        FULL OUTER JOIN 
            taxi_b 
            ON taxi_a.tpep_dropoff_datetime = taxi_b.tpep_dropoff_datetime
            '''
bc.sql(query)

#### FULL OUTER JOIN MULTI-COLUMN

In [ ]:
query = '''
        select 
            taxi_a.tpep_pickup_datetime, 
            taxi_b.passenger_count, 
            taxi_b.fare_amount 
        from 
            taxi_a 
        FULL OUTER JOIN 
            taxi_b 
            ON taxi_a.tpep_dropoff_datetime = taxi_b.tpep_dropoff_datetime AND taxi_a.tpep_pickup_datetime = taxi_b.tpep_pickup_datetime
            '''
bc.sql(query)

#### LEFT OUTER JOIN
[Docs](https://docs.blazingdb.com/docs/queries#left-outer) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#LEFT-OUTER-JOIN)

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi_a', '../data/sample_taxi.parquet')
bc.create_table('taxi_b', '../data/sample_taxi.csv', header=0)

#### LEFT JOIN SINGLE COLUMN

In [ ]:
query = '''
        select 
            taxi_a.tpep_pickup_datetime, 
            taxi_b.passenger_count, 
            taxi_b.fare_amount 
        from 
            taxi_a 
        LEFT JOIN 
            taxi_b 
            ON taxi_a.tpep_dropoff_datetime = taxi_b.tpep_dropoff_datetime
            '''
bc.sql(query)

#### LEFT JOIN MULTI-COLUMN

In [ ]:
query = '''
        select 
            taxi_a.tpep_pickup_datetime, 
            taxi_b.passenger_count, 
            taxi_b.fare_amount 
        from 
            taxi_a 
        LEFT JOIN 
            taxi_b 
            ON taxi_a.tpep_dropoff_datetime = taxi_b.tpep_dropoff_datetime AND taxi_a.tpep_pickup_datetime = taxi_b.tpep_pickup_datetime
            '''
bc.sql(query)

### ORDER BY
[Docs](https://docs.blazingdb.com/docs/queries#order-by) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#ORDER-BY)

```sql
SELECT column_A, column_B FROM table_A ORDER BY column_A
```

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

#### ORDER BY without Specificying (defaults to Ascending)

In [ ]:
bc.sql('select passenger_count, fare_amount FROM taxi ORDER BY trip_distance')

#### ORDER BY Specifying Ascending

In [ ]:
bc.sql('select passenger_count, fare_amount from taxi ORDER BY trip_distance ASC')

#### ORDER BY Specifying Descending

In [ ]:
bc.sql('select passenger_count, fare_amount from taxi ORDER BY trip_distance DESC')

### NESTED QUERIES
[Docs](https://docs.blazingdb.com/docs/queries#nested-queries) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#NESTED-QUERIES)

A query nested inside another query that uses values from the outer query may appear elsewhere. 

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

In [ ]:
query = '''
        select 
            pickup_time, passenger_count, trip_distance, cost 
        from
            (
            SELECT 
                *,
                CAST(tpep_pickup_datetime || '.0' AS timestamp) AS pickup_time, 
                total_amount - tip_amount AS cost 
            FROM 
                taxi 
                WHERE
                    passenger_count <> 1
            )
            '''
bc.sql(query)

In [ ]:
query = '''
        select 
            year(pickup_time) as pickup_year, month(pickup_time) as pickup_month, dayofmonth(pickup_time) as pickup_day,
            passenger_count, trip_distance, cost 
        from
            (
            SELECT 
                *,
                CAST(tpep_pickup_datetime || '.0' AS timestamp) AS pickup_time, 
                total_amount - tip_amount AS cost 
            FROM 
                taxi 
                WHERE
                    passenger_count <> 1
            )
            '''
bc.sql(query)

### CASE
[Docs](https://docs.blazingdb.com/docs/queries#case) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#CASE)

```sql
SELECT CASE WHEN column_A > 5 THEN 1 ELSE 0 END FROM table_A
```

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

#### CASE -- Single Filter

In [ ]:
query = '''
        select 
            CASE 
                WHEN fare_amount > 20 THEN 1 
                ELSE 0 END
        from 
            taxi
            '''
bc.sql(query)

#### CASE -- Multiple Filters

In [ ]:
query = '''
        select 
            CASE 
                WHEN fare_amount > 20 THEN 1 
                ELSE 0 END, 
            CASE 
                WHEN tpep_pickup_datetime like '%-01-01%' THEN 1 
                ELSE 0 END
        from
            taxi
            '''
bc.sql(query)

### CAST
[Docs](https://docs.blazingdb.com/docs/queries#cast) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#CAST)

```sql
SELECT CAST(column_A AS type) FROM table_A
```

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

#### CAST -- Single Filter

In [ ]:
query = '''
        select 
            CAST(fare_amount AS int) 
        from 
            taxi
            '''
bc.sql(query)

#### CAST -- Multiple Filters

In [ ]:
query = '''
        select 
            CAST(passenger_count AS float) passenger_count_float, 
            CAST(fare_amount AS int) fare_amount_int
        from
            taxi
            '''
bc.sql(query)

#### CAST after WHERE

In [ ]:
query = '''
        select
            *
        from
            taxi
            WHERE
                DAYOFMONTH(CAST(tpep_pickup_datetime || '.0' AS TIMESTAMP)) <= 7
                '''
bc.sql(query)

### LIKE
[Docs](https://docs.blazingdb.com/docs/queries#like) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#LIKE)

```sql
SELECT * FROM table_A WHERE column_A LIKE '%foo%'
```

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

#### LIKE -- Single Filter

In [ ]:
bc.sql("select * from taxi WHERE tpep_pickup_datetime LIKE '%15-01%'")

#### LIKE -- Multiple Filters (AND)

In [ ]:
query = '''
        select    
            * 
        from 
            taxi 
            WHERE 
                tpep_pickup_datetime LIKE '%01-01%' 
                AND tpep_dropoff_datetime LIKE '%01-01%'
                '''
bc.sql(query)

#### LIKE -- Multiple Filters (OR)

In [ ]:
query = '''
        select 
            * 
        from 
            taxi 
        WHERE 
            tpep_pickup_datetime LIKE '%01-01%' 
            OR tpep_dropoff_datetime LIKE '%01-01%'
            '''
bc.sql(query)

#### LIKE -- Multiple Filters (AND/OR)

In [ ]:
query = '''
        select 
            * 
        from 
            taxi 
        WHERE 
            (tpep_pickup_datetime LIKE '%01-01%' OR tpep_dropoff_datetime LIKE '%01-01%') 
            AND dropoff_x LIKE '-822%'
            '''
bc.sql(query)

.


.


.

## Functions

### Unary Functions
[Docs](https://docs.blazingdb.com/docs/unary-ops) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#Unary-Functions)

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

#### FLOOR
Returns the largest integer value that is smaller than or equal to a number.
```sql
SELECT FLOOR(column_A) FROM table_A
```

In [ ]:
bc.sql('select FLOOR(trip_distance), trip_distance from taxi')

#### CEILING
Returns the smallest integer value that is larger than or equal to a number.
```sql
SELECT CEILING(column_A) FROM table_A
```

In [ ]:
bc.sql('select CEILING(trip_distance), trip_distance from taxi')

#### SIN
```sql
SELECT SIN(column_A) FROM table_A
```

In [ ]:
bc.sql('select SIN(trip_distance) from taxi')

#### COS
```sql
SELECT COS(column_A) FROM table_A
```

In [ ]:
bc.sql('select COS(trip_distance) from taxi')

#### TAN
```sql
SELECT TAN(column_A) FROM table_A
```

In [ ]:
bc.sql('select TAN(trip_distance) from taxi')

#### ASIN

In [ ]:
bc.sql('select ASIN(trip_distance) from taxi')

#### ACOS

In [ ]:
bc.sql('select ACOS(trip_distance) from taxi')

#### ATAN

In [ ]:
bc.sql('select ATAN(trip_distance) from taxi')

#### ABSOLUTE (ABS)

In [ ]:
bc.sql('select ABS(dropoff_x) from taxi')

#### LN

In [ ]:
bc.sql('select LN(trip_distance) from taxi')

#### LOG

In [ ]:
bc.sql('select LOG10(trip_distance) from taxi')

.


.


.

### Binary Functions
[Docs](https://docs.blazingdb.com/docs/binary-ops) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#Binary-Functions)

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

#### Addition

In [ ]:
bc.sql('select fare_amount + tip_amount from taxi')

#### Subtraction

In [ ]:
bc.sql('select total_amount - fare_amount from taxi')

#### Multiplication

In [ ]:
bc.sql('select trip_distance * passenger_count from taxi')

#### Division

In [ ]:
bc.sql('select tip_amount / passenger_count from taxi')

#### MODULUS (MOD)

In [ ]:
bc.sql('select MOD(tip_amount, fare_amount) from taxi')

#### POWER

In [ ]:
bc.sql('select POWER(tip_amount, fare_amount) from taxi')

#### EQUAL TO

In [ ]:
bc.sql('select (fare_amount = total_amount) from taxi')

#### NOT EQUAL TO

In [ ]:
bc.sql('select fare_amount <> total_amount from taxi')

#### LESS THAN

In [ ]:
bc.sql('select fare_amount < tip_amount from taxi')

#### GREATER THAN

In [ ]:
bc.sql('select fare_amount > tip_amount from taxi')

#### LESS THAN OR EQUAL TO

In [ ]:
bc.sql('select fare_amount <= tip_amount from taxi')

#### GREATER THAN OR EQUAL TO

In [ ]:
bc.sql('select fare_amount >= tip_amount from taxi')

.


.


.

### DateTime Functions
[Docs](https://docs.blazingdb.com/docs/datetime-functions) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#DateTime-Functions)

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

#### YEAR

In [ ]:
query = '''
        select 
            YEAR(
                cast(tpep_pickup_datetime || '.0' as timestamp)
                )
        from
            taxi
        '''
bc.sql(query)

#### MONTH

In [ ]:
query = '''
        select 
            MONTH(
                cast(tpep_pickup_datetime || '.0' as timestamp)
                )
        from
            taxi
        '''
bc.sql(query)

#### DAY

In [ ]:
query = '''
        select 
            DAYOFMONTH(
                cast(tpep_pickup_datetime || '.0' as timestamp)
                )
        from
            taxi
        '''
bc.sql(query)

#### HOUR

In [ ]:
query = '''
        select 
            HOUR(
                cast(tpep_pickup_datetime || '.0' as timestamp)
                )
        from
            taxi
        '''
bc.sql(query)

#### MINUTE

In [ ]:
query = '''
        select 
            MINUTE(
                cast(tpep_pickup_datetime || '.0' as timestamp)
                )
        from
            taxi
        '''
bc.sql(query)

#### SECOND

In [ ]:
query = '''
        select 
            SECOND(
                cast(tpep_pickup_datetime || '.0' as timestamp)
                )
        from
            taxi
        '''
bc.sql(query)

.


.


.

### String Functions
[Docs](https://docs.blazingdb.com/docs/string-functions) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#String-Functions)

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

#### CONCAT
Concatenate two strings with the `||` operator.

```sql
SELECT column_A || column_B FROM table_A
```

In [ ]:
query = '''
        select 
            tpep_pickup_datetime || '.001'
        from
            taxi
            '''
bc.sql(query)

#### SUBSTRING
```sql
SELECT SUBSTRING(string_column, start, length) FROM table
```

In [ ]:
query = '''
        select 
            SUBSTRING(tpep_pickup_datetime, 0, 4)
        from
            taxi
            '''
bc.sql(query)

In [ ]:
query = '''
        select 
            SUBSTRING(tpep_pickup_datetime, 0, 4) || '-' || SUBSTRING(tpep_pickup_datetime, 12, 8) 
        from
            taxi
            '''
bc.sql(query)

#### CHAR_LENGTH
Gets the length of a string.
```sql
SELECT CHAR_LENGTH(string_col) FROM table
```

In [ ]:
query = '''
        select 
            CHAR_LENGTH(tpep_pickup_datetime) 
        from 
            taxi
            '''
bc.sql(query)

.


.


.

## Reserved Words
[Docs](https://docs.blazingdb.com/docs/reserved-words) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#String-Functions)

BlazingSQL uses the Apache Calcite industry-standard SQL parser, validator and JDBC driver for SQL language. There is a common list of words that are reserved for the engine and, outside of their specified functionality, should not be used by users. 

Exmples:
- date
- from
- time
- select

You can find this list in:
- [Calcite Docs](https://calcite.apache.org/docs/reference.html#keywords)
- [Calcite Source Code](https://github.com/apache/calcite/blob/master/core/src/main/codegen/templates/Parser.jj#L6740)

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()
bc.create_table('taxi', '../data/sample_taxi.parquet')

In [ ]:
query = '''
        select
            tpep_pickup_datetime as time
        from
            taxi
            '''
bc.sql(query)

.


.


.